In [1]:
!pip install pyannote.audio

In [2]:
!pip install torchvision

In [3]:
! pip install python-dotenv
! pip install pydub
! pip install pandas
! pip install psycopg2
! pip install google-auth google-auth-httplib2 google-api-python-client google-auth-oauthlib

In [4]:
import sys
sys.path.append('../util')

from util import read_spreadsheet
from util import collect_segments
from util import split_audio_files

Lightning automatically upgraded your loaded checkpoint from v1.1.3 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/059e96f964841d40f1a5e755bb7223f76666bba4/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.7.1, yours is 2.4.0+cu121. Bad things might happen unless you revert torch to 1.x.


In [5]:
import requests
import os
import subprocess

def download_url_file(url, local_filename):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_filename.strip(), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded {local_filename.strip()}")
    else:
        print(f"Failed to download {local_filename.strip()} from {url}. Status code: {response.status_code}")


In [ ]:
from pydub import AudioSegment
import pandas as pd

# Function to get audio duration from an audio file
def get_audio_duration(audio_filename):
    try:
        audio = AudioSegment.from_file(audio_filename)
        duration_in_sec = len(audio) // 1000  # Duration in seconds
        duration = pd.to_datetime(duration_in_sec, unit='s').strftime('%H:%M:%S')  # Duration in H:M:S format
        return duration
    except Exception as e:
        print(f"Error fetching duration for {audio_filename}: {e}")
        return None


In [6]:
# Ensure directories exist
os.makedirs('audio_from_url', exist_ok=True)

# Example usage:
from_id = 163 
to_id = 201

durations = []

df = read_spreadsheet(sheet_id="1SjlnWsr6YVhBIW6EaS0ITmboDmEuj2VnxGRNL50OPx4")

for index, row in df.iterrows():
    if not isinstance(row['ID'], str) or not isinstance(row['YouTube link'], str):
        break
    id = row['ID']
    url_path = row['YouTube link']
    audio_filename = f"audio_from_url/{id}.mp3".strip()
    sr_no = row.iloc[0]
    if sr_no >= from_id and sr_no <= to_id:
        if os.path.exists(audio_filename):
            print(f"Audio file {audio_filename} already exists. Skipping extraction.")
            continue
        if not os.path.exists(audio_filename):
            download_url_file(url_path, audio_filename)
        duration = get_audio_duration(audio_filename)
        print(f"Audio duration for {audio_filename}: {duration}")
        durations.append((id, duration))



Downloaded audio_from_url/STT_HS0163.mp3
Downloaded audio_from_url/STT_HS0164.mp3
Downloaded audio_from_url/STT_HS0165.mp3
Downloaded audio_from_url/STT_HS0166.mp3
Downloaded audio_from_url/STT_HS0167.mp3
Downloaded audio_from_url/STT_HS0168.mp3
Downloaded audio_from_url/STT_HS0169.mp3
Downloaded audio_from_url/STT_HS0170.mp3
Downloaded audio_from_url/STT_HS0171.mp3
Downloaded audio_from_url/STT_HS0172.mp3
Downloaded audio_from_url/STT_HS0173.mp3
Downloaded audio_from_url/STT_HS0174.mp3
Downloaded audio_from_url/STT_HS0175.mp3
Downloaded audio_from_url/STT_HS0176.mp3
Downloaded audio_from_url/STT_HS0177.mp3
Downloaded audio_from_url/STT_HS0178.mp3
Downloaded audio_from_url/STT_HS0179.mp3
Downloaded audio_from_url/STT_HS0180.mp3
Downloaded audio_from_url/STT_HS0181.mp3
Downloaded audio_from_url/STT_HS0182.mp3
Downloaded audio_from_url/STT_HS0183.mp3
Downloaded audio_from_url/STT_HS0184.mp3
Downloaded audio_from_url/STT_HS0185.mp3
Downloaded audio_from_url/STT_HS0186.mp3
Downloaded audio

In [ ]:
# Convert list to DataFrame and save to CSV
df_durations = pd.DataFrame(durations, columns=['File Name', 'Duration'])
df_durations.to_csv('nw_gh_audio_durations.csv', index=False)

print("Durations have been saved to audio_durations.csv")